In [2]:
#Load the MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
#import tensorflow
import tensorflow as tf
# start a tf interactive session. a session (usually denoted as sess) is tf's connection to c++ background
# interactive sessions are useful for enviornments like ipython, where you don't execute the whole script at once
sess = tf.InteractiveSession()


In [4]:
#build a softmax regression model
#step 1: set up your placeholder variables
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])
# the mnist pics are 28x28 = 784 pixels, which is the length of floating points in the image. 
# none corresponds to batch size, or the sample of your x values being run through an epoch.
# y_ has a shape of 10 because there will be 10 outputs corresponding to the probability of each number.

In [5]:
#build your variables. W and b (weights and bias) are important. They can be initialized
#as zero because we will 'learn' W and b.
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))
# the weights will be a shape of 784 x 10 because we have 784 input features and 10 outputs
# b is a 10-dimensional vector (because we have 10 classes), and b is a matrix of biases per class.

In [6]:
#initialize all variables
sess.run(tf.initialize_all_variables())

In [7]:
#predicted class and cost function
y = tf.nn.softmax(tf.matmul(x,W) + b)
#We multiply the vectorized input images x by the weight matrix W, 
#add the bias b, and compute the softmax probabilities that are assigned to each class

In [8]:
#use cross-entropy as your cost function here
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
# tf.reduce_sum sums across all classes and tf.reduce_mean takes the average over these sums.

In [9]:
# train the model
# set up your training step
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
#The returned operation train_step, when run, will apply the gradient descent updates to the parameters. 
#Training the model can therefore be accomplished by repeatedly running train_step.

In [10]:
for i in range(1000):
  batch = mnist.train.next_batch(50)
  train_step.run(feed_dict={x: batch[0], y_: batch[1]})

In [11]:
#evaluate the model
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
# tf.argmax is an extremely useful function which gives you the index of the highest entry 
# in a tensor along some axis.
# [True, False, True, True] would become [1,0,1,1] which would become 0.75.

In [12]:
# get the model accuracy
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9092


# Build a Multilayer Convolutional Network

Getting 91% accuracy on MNIST is bad. It's almost embarrassingly bad. In this section, we'll fix that, jumping from a very simple model to something moderately sophisticated: a small convolutional neural network. This will get us to around 99.2% accuracy -- not state of the art, but respectable.

## 1. Weight Initialization

To create this model, we're going to need to create a lot of weights and biases. One should generally initialize weights with a small amount of noise for symmetry breaking, and to prevent 0 gradients. Since we're using ReLU neurons, it is also good practice to initialize them with a slightly positive initial bias to avoid "dead neurons." Instead of doing this repeatedly while we build the model, let's create two handy functions to do it for us.

In [13]:
# initializes the weight of a neuron with a slightly positive value from a truncated normal
# distribution with a stddev of .1
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

# initializes the bias of a class similarly as above for neurons
def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

## 2. Convolution and Pooling

TensorFlow also gives us a lot of flexibility in convolution and pooling operations. How do we handle the boundaries? What is our stride size? In this example, we're always going to choose the vanilla version. Our convolutions uses a stride of one and are zero padded so that the output is the same size as the input. Our pooling is plain old max pooling over 2x2 blocks. To keep our code cleaner, let's also abstract those operations into functions.

In [14]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

### 2a. First Convolutional Layer

We can now implement our first layer. It will consist of convolution, followed by max pooling. The convolutional will compute 32 features for each 5x5 patch. Its weight tensor will have a shape of [5, 5, 1, 32]. The first two dimensions are the patch size, the next is the number of input channels, and the last is the number of output channels. We will also have a bias vector with a component for each output channel.

A convoluted layer is a hidden layer, and convoluted means functions layered on functions. Pooling is spatial binning of pixels, which reduces the 'roughness' of object's edges. You can do min or max pooling.

In [15]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
# W: [patch width, patch height, number of input channels, number of output channels]
# b: [1 bias for each output channel (class)] 

To apply the layer, we first reshape x to a 4d tensor, with the second and third dimensions corresponding to image width and height, and the final dimension corresponding to the number of color channels.

In [16]:
x_image = tf.reshape(x, [-1,28,28,1])
# I think the first dimension's depth, so it would be [depth, width , height, channels]

We then convolve x_image with the weight tensor, add the bias, apply the ReLU function, and finally max pool.



In [17]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

### 2b. Second Convolutional Layer

In order to build a deep network, we stack several layers of this type. The second layer will have 64 features for each 5x5 patch.

In [18]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

### 2c. Densely Connected Layer

Now that the image size has been reduced to 7x7, we add a fully-connected layer with 1024 neurons to allow processing on the entire image. We reshape the tensor from the pooling layer into a batch of vectors, multiply by a weight matrix, add a bias, and apply a ReLU.

In [19]:
# fc is fully-connected. Takes in the
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

### 2d. Dropout

To reduce overfitting, we will apply dropout before the readout layer. We create a placeholder for the probability that a neuron's output is kept during dropout. This allows us to turn dropout on during training, and turn it off during testing. TensorFlow's tf.nn.dropout op automatically handles scaling neuron outputs in addition to masking them, so dropout just works without any additional scaling.

In [20]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

### 2e. Readout Layer

Finally, we add a softmax layer, just like for the one layer softmax regression above.

In [21]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

## 3. Train and Evaluate the Model

How well does this model do? To train and evaluate it we will use code that is nearly identical to that for the simple one layer SoftMax network above. The differences are that: we will replace the steepest gradient descent optimizer with the more sophisticated ADAM optimizer; we will include the additional parameter keep_prob in feed_dict to control the dropout rate; and we will add logging to every 100th iteration in the training process.

In [22]:
# define the cost function, cross_entropy in this case
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))

# The train step here uses AdamOptimizer not gradient descent
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# just like above
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.initialize_all_variables())
for i in range(300):
  batch = mnist.train.next_batch(50)
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1], keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))
  train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.06
step 100, training accuracy 0.84
step 200, training accuracy 0.98
test accuracy 0.9194


When you let this run to 20,000 epochs, the final test set accuracy after running this code should be approximately 99.2%.

We have learned how to quickly and easily build, train, and evaluate a fairly sophisticated deep learning model using TensorFlow.

1: For this small convolutional network, performance is actually nearly identical with and without dropout. Dropout is often very effective at reducing overfitting, but it is most useful when training very large neural networks. 

In [56]:
# save the model so you can continue from the current spot
saver = tf.train.Saver()
saver.save(sess, 'deep-mnist-1', global_step=300)
# use saver.restore('deep-mnist-1', save_path)

'deep-mnist-1-300'

In [70]:
!ls

MNIST w JAM.ipynb           deep MNIST with notes.ipynb
MNIST_data                  deep-mnist-1-300
Untitled.ipynb              deep-mnist-1-300.meta
checkpoint                  tf1.py


In [60]:
!pwd

/Users/michaelgfrantz/Dropbox/General Assembly/Capstone/Practice


In [71]:
import tensorflow.contrib.learn as skflow

784